# Task 2: Outlier detection

In [1]:
# install.packages('gdata')     # to read in spreadsheets
# install.packages('dplyr')     # filtering, mapping and aggregating functionality
# install.packages('Rmisc')     # data formatting

In [2]:
library(readxl)   # read in spreadsheet
library(Rmisc)    # getting rid of NA values
library(dplyr)    # pipelining operations
library(reshape2) # used for clearing data points of NA values


Loading required package: lattice
Loading required package: plyr

Attaching package: 'dplyr'

The following objects are masked from 'package:plyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



# Just for reference: check crowd statistics before any removals

In [3]:
#load data
speech_quality <- read_excel("./datasets/DB04_speech_quality_crowdsourcing_dataset.xlsx", sheet = 3)
speech_quality$cond = substr(speech_quality$files,6,8) 

In [4]:
# read in crowd with condition and ratings
p <- select(speech_quality, starts_with("P"), cond)
crowd <- melt(p, id.vars=c('cond'))
crowd <- na.omit(crowd)
# grouped by condition, get necessary statistics from crowd

crowd_stats <- crowd %>% 
             group_by(cond) %>% 
             summarise(Mean = mean(value, na.rm = TRUE), `Standard Deviation` = sd(value, na.rm = TRUE), n = n()) %>% 
             mutate(CI95=`Standard Deviation`/sqrt(n) * qt(0.95, df=n-1))

crowd_stats %>%
            rename(Condition = cond) %>% 
            mutate(Condition = as.integer(substring(Condition,2,3)))

Condition,Mean,Standard Deviation,n,CI95
1,4.836449,0.4294100,214,0.04849373
2,1.541284,0.5844705,218,0.06539125
3,4.351852,0.8048090,216,0.09046249
4,2.783410,0.9149486,217,0.10260316
5,3.537037,0.7335917,216,0.08245750
6,4.623256,0.5489315,215,0.06184582
7,3.665094,1.0377283,212,0.11774812
8,3.933962,0.7255509,212,0.08232622
9,3.117371,0.8579971,213,0.09712371
10,4.518182,0.6443547,220,0.07175992


# Boxplot outlier detection

In [5]:
# read in CS_Full and conditions
cs_full_box <- read_excel("datasets/DB04_speech_quality_crowdsourcing_dataset.xlsx", sheet = 2)
cs_full_box$cond = substr(cs_full_box$files,6,8)

cs_full_box_global <- cs_full_box # copy for way 2 because we are not sure whether we understood the task correctly

In [6]:
# clear CS_Full of NA cells

a <- select(cs_full_box, starts_with("A"), cond)   # get workers
workerfull <- melt(a, id.vars=c('cond'))       # merge workers and condition
workerfull <- na.omit(workerfull)              # omit NA cells

In [7]:
# compute statistics per condition as requested
IQRcrowd <- workerfull %>% 
                group_by(cond) %>% 
                summarize(lower = quantile(value, c(0.25)), upper = quantile(value, c(0.75))) %>% 
                mutate(IQR = upper - lower) %>% 
                mutate(lower_bound = lower - 1.5*IQR, upper_bound = upper + 1.5*IQR)

outlier <- data.frame()

# find individual outliers
for (i in (1:nrow(workerfull))){ # for all rows
    cond <- filter(IQRcrowd, workerfull[i,"cond"] == IQRcrowd[1]) # get condition
    # check if rating is outlier
    if (workerfull[i,"value"] < cond["lower_bound"] || workerfull[i,"value"] > cond["upper_bound"]){ 
        outlier[nrow(outlier)+1,"cond"] <- (unlist(workerfull[i,"cond"]))                   
        outlier[nrow(outlier),"variable"] <- (unlist(workerfull[i,"variable"]))
        outlier[nrow(outlier),"value"] <- (unlist(workerfull[i,"value"]))
    }
}

outlier

cond,variable,value
c01,A10DEO061A6L3O,4
c34,A1AJWS2H14J8MQ,1
c22,A1CA46R2A6TV9W,5
c33,A1CGDMOPDIC7EU,1
c01,A1IFIK8J49WBER,4
c09,A1IFIK8J49WBER,1
c48,A1MJVTR0PCKBWW,1
c23,A1V2H0UF94ATWY,1
c34,A1V2H0UF94ATWY,1
c01,A1X1IRFYQZ40LD,3


# Boxplot outlier detection - Way 1: Workers to be removed = Workers with more than two univariate outliers ** per condition ** 

In [8]:
# find workers with more than 2 univariate outliers per condition
outliers <- filter(outlier %>% count(cond, variable), n > 2)

outliers

cond,variable,n
c01,A258MR1IS96JEP,3
c01,A2NGMLBFZ3YQP5,6
c01,A3AJJHOAV7WIUQ,3
c03,A2NGMLBFZ3YQP5,3
c07,A2NGMLBFZ3YQP5,4
c15,A2FYQIZWV2UJHS,3
c25,A2NGMLBFZ3YQP5,3
c27,AZLZA0Q87TJZO,3
c46,A2FYQIZWV2UJHS,3


In [9]:
# remove workers with more than 2 outliers outliers per condition completely from CS_Full
for (var in outliers$variable){
    cs_full_box[, var] <- NA
}



# Boxplot outlier detection - Way 1: Solutions per condition and file 

In [10]:
#results per condition
p <- select(cs_full_box, starts_with("A"), cond)
cs_full_box_c1 <- melt(p, id.vars=c('cond'))
cs_full_box_c1 <- na.omit(cs_full_box_c1)

cs_full_box_cond <- cs_full_box_c1 %>% 
               group_by(cond) %>% 
               summarise(Mean = mean(value, na.rm = TRUE), `Standard Deviation` = sd(value, na.rm = TRUE), n = n()) %>% 
               mutate(CI95=`Standard Deviation`/sqrt(n) * qt(0.95, df=n-1))  %>%
               rename(Condition = cond) %>% 
               mutate(Condition = as.integer(substring(Condition,2,3)))

cs_full_box_cond

Condition,Mean,Standard Deviation,n,CI95
1,4.882979,0.3829629,188,0.04617022
2,1.556122,0.5921826,196,0.06990738
3,4.443243,0.7285112,185,0.08854616
4,2.837696,0.8644091,191,0.10338383
5,3.610811,0.6592914,185,0.08013291
6,4.649746,0.5384809,197,0.06340478
7,3.777778,0.9693897,189,0.11655747
8,4.000000,0.7108983,187,0.08593744
9,3.181818,0.8418432,187,0.10176680
10,4.574468,0.5756068,188,0.06939547


In [11]:
#results per file
p <- select(cs_full_box, starts_with("A"), files)
cs_full_box_f1 <- melt(p, id.vars=c('files'))
cs_full_box_f1 <- na.omit(cs_full_box_f1)

cs_full_box_file <- cs_full_box_f1 %>% 
                       group_by(files) %>% 
                       summarise(Mean = mean(value, na.rm = TRUE), `Standard Deviation` = sd(value, na.rm = TRUE), n = n()) %>% 
                       mutate(CI95=`Standard Deviation`/sqrt(n) * qt(0.95, df=n-1))  %>%
                       rename(File = files)

cs_full_box_file

File,Mean,Standard Deviation,n,CI95
D401_c01_ef01_s001.wav,4.875000,0.3535534,8,0.2368223
D401_c01_ef01_s002.wav,4.875000,0.3535534,8,0.2368223
D401_c01_ef01_s003.wav,4.888889,0.3333333,9,0.2066164
D401_c01_ef01_s004.wav,4.857143,0.3779645,7,0.2775972
D401_c01_ef02_s001.wav,4.857143,0.3779645,7,0.2775972
D401_c01_ef02_s002.wav,5.000000,0.0000000,6,0.0000000
D401_c01_ef02_s003.wav,5.000000,0.0000000,6,0.0000000
D401_c01_ef02_s004.wav,4.888889,0.3333333,9,0.2066164
D401_c01_ef03_s001.wav,4.888889,0.3333333,9,0.2066164
D401_c01_ef03_s002.wav,5.000000,0.0000000,7,0.0000000


# Boxplot outlier detection - Way 2: Workers to be removed = Workers with more than two univariate outliers ** globally **

In [12]:
# find workers with more than 2 univariate outliers globally

outliers_box_global <- outlier %>% group_by(variable) %>% summarise(n = n()) %>% filter(n > 2)
outliers_box_global

variable,n
A1X1IRFYQZ40LD,4
A23TGEQ4CG90PA,4
A258MR1IS96JEP,5
A27PBC5O3Z5ZED,4
A2FYQIZWV2UJHS,13
A2G0HRT8HS9HRZ,3
A2IQ0QCTQ3KWLT,10
A2NGMLBFZ3YQP5,27
A2P76QVLSGJR45,5
A2TI9CGVG147MZ,8


In [13]:
# remove workers with more than 2 outliers globally
for (var in outliers_box_global $variable){
    cs_full_box_global[, var] <- NA
}

# Boxplot outlier detection - Way 2: Solutions per condition and file

In [14]:
#results per condition
p <- select(cs_full_box_global, starts_with("A"), cond)
cs_full_box_c2 <- melt(p, id.vars=c('cond'))
cs_full_box_c2 <- na.omit(cs_full_box_c2)

cs_full_box_cond_2 <- cs_full_box_c2 %>% 
               group_by(cond) %>% 
               summarise(Mean = mean(value, na.rm = TRUE), `Standard Deviation` = sd(value, na.rm = TRUE), n = n()) %>% 
               mutate(CI95=`Standard Deviation`/sqrt(n) * qt(0.95, df=n-1)) %>%
               rename(Condition = cond) %>% 
               mutate(Condition = as.integer(substring(Condition,2,3)))

cs_full_box_cond_2

Condition,Mean,Standard Deviation,n,CI95
1,4.916667,0.2783250,72,0.05466599
2,1.666667,0.6036067,87,0.10760326
3,4.378049,0.7476753,82,0.13738178
4,2.687500,0.7855605,96,0.13317637
5,3.607143,0.7116561,84,0.12916132
6,4.650000,0.5000000,100,0.08301956
7,3.793103,0.8910137,87,0.15883848
8,3.880435,0.6435204,92,0.11149095
9,3.142857,0.7783649,84,0.14126858
10,4.476744,0.5886524,86,0.10555925


In [15]:
p <- select( cs_full_box_global, starts_with("A"), files)
cs_full_box_f2 <- melt(p, id.vars=c('files'))
cs_full_box_f2 <- na.omit(cs_full_box_f2)

cs_full_box_files2 <-  cs_full_box_f2 %>% 
                  group_by(files) %>% 
                  summarise(Mean = mean(value, na.rm = TRUE), `Standard Deviation` = sd(value, na.rm = TRUE), n = n()) %>% 
                  mutate(CI95=`Standard Deviation`/sqrt(n) * qt(0.95, df=n-1)) %>%
                  rename(File = files)
cs_full_box_files2

Warning message in qt(0.95, df = n - 1):
"NaNs produced"

File,Mean,Standard Deviation,n,CI95
D401_c01_ef01_s001.wav,5.000000,0.0000000,2,0.0000000
D401_c01_ef01_s002.wav,5.000000,0.0000000,3,0.0000000
D401_c01_ef01_s003.wav,5.000000,0.0000000,3,0.0000000
D401_c01_ef01_s004.wav,4.000000,NaN,1,NaN
D401_c01_ef02_s001.wav,5.000000,0.0000000,3,0.0000000
D401_c01_ef02_s002.wav,5.000000,0.0000000,4,0.0000000
D401_c01_ef02_s003.wav,5.000000,0.0000000,2,0.0000000
D401_c01_ef02_s004.wav,4.666667,0.5773503,3,0.9733285
D401_c01_ef03_s001.wav,4.750000,0.5000000,4,0.5883409
D401_c01_ef03_s002.wav,5.000000,0.0000000,3,0.0000000


# Z-score outlier detection

In [16]:
# read in CS_Full and conditions
cs_full_z <- read_excel("./datasets/DB04_speech_quality_crowdsourcing_dataset.xlsx", sheet = 2)
cs_full_z$cond = substr(cs_full_z$files,6,8)

cs_full_z_2 <- cs_full_z # copy for the other part because we are not sure whether we understood the task correctly

In [17]:
# clear CS_Full of NA cells

a <- select(cs_full_z, starts_with("A"), cond)     # get workers
workerfull <- melt(a, id.vars=c('cond'))           # merge workers and condition
workerfull <- na.omit(workerfull)                  # omit NA cells

In [18]:
outlier <- data.frame()

for (i in (1:nrow(workerfull))){
    #just the row of crowd_stats where condition matches, crowd_stats holds means, sd and ci
    cond <- filter(crowd_stats, workerfull[i,"cond"] == crowd_stats["cond"])
    #abs etc is the z score
    if(abs((workerfull[i,"value"]-cond["Mean"])/cond["Standard Deviation"]) > 3.29){
        #append to outlier
        outlier[nrow(outlier)+1,"cond"] <- (unlist(workerfull[i,"cond"]))
        outlier[nrow(outlier),"variable"] <- (unlist(workerfull[i,"variable"]))
        outlier[nrow(outlier),"value"] <- (unlist(workerfull[i,"value"]))
    }
}

outlier

cond,variable,value
c01,A1X1IRFYQZ40LD,3
c17,A1X1IRFYQZ40LD,1
c25,A1X1IRFYQZ40LD,1
c01,A23TGEQ4CG90PA,2
c17,A27PBC5O3Z5ZED,1
c24,A27PBC5O3Z5ZED,4
c13,A2FYQIZWV2UJHS,2
c17,A2FYQIZWV2UJHS,1
c47,A2FYQIZWV2UJHS,2
c17,A2IQ0QCTQ3KWLT,1


# Z-Score outlier detection - Way 1: Workers to be removed = Workers with more than two univariate outliers ** per condition **

In [19]:
# find workers with more than 2 univariate outliers per condition
outliers_z <- filter(outlier %>% count(cond, variable), n > 2)
outliers_z

cond,variable,n


In [20]:
# remove workers with more than 2 univariate outliers outliers completely from CS_Full
for (var in outliers_z$variable){
    cs_full_z[, var] <- NA
}

# Z-Score outlier detection - Way 1: Solutions per condition and file

In [21]:
#results per condition
p <- select( cs_full_z, starts_with("A"), cond)
cs_full_z_c <- melt(p, id.vars=c('cond'))
cs_full_z_c <- na.omit(cs_full_z_c)

cs_full_z_cond <-  cs_full_z_c %>% 
                  group_by(cond) %>% 
                  summarise(Mean = mean(value, na.rm = TRUE), `Standard Deviation` = sd(value, na.rm = TRUE), n = n()) %>% 
                  mutate(CI95=`Standard Deviation`/sqrt(n) * qt(0.95, df=n-1)) %>%
                  rename(Condition = cond) %>%
                  mutate(Condition = as.integer(substring(Condition, 2, 3)))

cs_full_z_cond

Condition,Mean,Standard Deviation,n,CI95
1,4.836449,0.4294100,214,0.04849373
2,1.541284,0.5844705,218,0.06539125
3,4.351852,0.8048090,216,0.09046249
4,2.783410,0.9149486,217,0.10260316
5,3.537037,0.7335917,216,0.08245750
6,4.623256,0.5489315,215,0.06184582
7,3.665094,1.0377283,212,0.11774812
8,3.933962,0.7255509,212,0.08232622
9,3.117371,0.8579971,213,0.09712371
10,4.520548,0.6448723,219,0.07198274


In [22]:
#results per file
p <- select(cs_full_z, starts_with("A"), files)
cs_full_z_f <- melt(p, id.vars=c('files'))
cs_full_z_f <- na.omit(cs_full_z_f)

cs_full_z_files <- cs_full_z_f %>% 
                       group_by(files) %>% 
                       summarise(Mean = mean(value, na.rm = TRUE), `Standard Deviation` = sd(value, na.rm = TRUE), n = n()) %>% 
                       mutate(CI95=`Standard Deviation`/sqrt(n) * qt(0.95, df=n-1)) %>%
                       rename(File = files)
cs_full_z_files

File,Mean,Standard Deviation,n,CI95
D401_c01_ef01_s001.wav,4.777778,0.4409586,9,0.2733279
D401_c01_ef01_s002.wav,4.777778,0.4409586,9,0.2733279
D401_c01_ef01_s003.wav,4.800000,0.4216370,10,0.2444151
D401_c01_ef01_s004.wav,4.857143,0.3779645,7,0.2775972
D401_c01_ef02_s001.wav,4.750000,0.4629100,8,0.3100732
D401_c01_ef02_s002.wav,5.000000,0.0000000,7,0.0000000
D401_c01_ef02_s003.wav,5.000000,0.0000000,8,0.0000000
D401_c01_ef02_s004.wav,4.888889,0.3333333,9,0.2066164
D401_c01_ef03_s001.wav,4.900000,0.3162278,10,0.1833113
D401_c01_ef03_s002.wav,5.000000,0.0000000,7,0.0000000


# Z-Score outlier detection - Way 2: Workers to be removed = Workers with more than two univariate outliers ** globally **

In [23]:
# find workers with more than 2 outliers globally

outliers_z_2 <- outlier %>% group_by(variable) %>% summarise(n = n()) %>% filter(n > 2) 
# delete the filter part if you want to see every worker which caused at least one outlier

outliers_z_2 

variable,n
A1X1IRFYQZ40LD,3
A2FYQIZWV2UJHS,3
A2NGMLBFZ3YQP5,4


In [24]:
# remove workers with more than 2 outliers globally
for (var in outliers_z_2 $variable){
    cs_full_z_2[, var] <- NA
}

# Z-Score outlier detection - Way 2: Solutions per condition and file

In [25]:
# results per condition
p <- select( cs_full_z_2, starts_with("A"), cond)
cs_full_z_2_c <- melt(p, id.vars=c('cond'))
cs_full_z_2_c <- na.omit(cs_full_z_2_c)

cs_full_z_2_cond <-  cs_full_z_2_c %>% 
                  group_by(cond) %>% 
                  summarise(Mean = mean(value, na.rm = TRUE), `Standard Deviation` = sd(value, na.rm = TRUE), n = n()) %>% 
                  mutate(CI95=`Standard Deviation`/sqrt(n) * qt(0.95, df=n-1)) %>%
                  rename(Condition = cond) %>%
                  mutate(Condition = as.integer(substring(Condition, 2, 3)))

cs_full_z_2_cond

Condition,Mean,Standard Deviation,n,CI95
1,4.870647,0.3783967,201,0.04410552
2,1.544554,0.5905517,202,0.06866189
3,4.441026,0.7324292,195,0.08668710
4,2.810000,0.8932466,200,0.10437828
5,3.617801,0.6847953,191,0.08190192
6,4.656863,0.5156678,204,0.05965799
7,3.744792,0.9881359,192,0.11787044
8,3.989899,0.7052371,198,0.08282795
9,3.178571,0.8435274,196,0.09957873
10,4.587940,0.5782727,199,0.06774389


In [26]:
# results per file
p <- select( cs_full_z_2, starts_with("A"), files)
cs_full_z_2_f <- melt(p, id.vars=c('files'))
cs_full_z_2_f <- na.omit(cs_full_z_2_f)
cs_full_z_2_files <-  cs_full_z_2_f %>% 
                  group_by(files) %>% 
                  summarise(means = mean(value, na.rm = TRUE), sds = sd(value, na.rm = TRUE), n = n()) %>% 
                  mutate(CI95=sds/sqrt(n) * qt(0.95, df=n-1)) %>%
                  rename(File = files)

cs_full_z_2_files

File,means,sds,n,CI95
D401_c01_ef01_s001.wav,4.777778,0.4409586,9,0.2733279
D401_c01_ef01_s002.wav,4.875000,0.3535534,8,0.2368223
D401_c01_ef01_s003.wav,4.800000,0.4216370,10,0.2444151
D401_c01_ef01_s004.wav,4.857143,0.3779645,7,0.2775972
D401_c01_ef02_s001.wav,4.750000,0.4629100,8,0.3100732
D401_c01_ef02_s002.wav,5.000000,0.0000000,7,0.0000000
D401_c01_ef02_s003.wav,5.000000,0.0000000,8,0.0000000
D401_c01_ef02_s004.wav,4.888889,0.3333333,9,0.2066164
D401_c01_ef03_s001.wav,4.900000,0.3162278,10,0.1833113
D401_c01_ef03_s002.wav,5.000000,0.0000000,7,0.0000000
